# Langchain Expression Language (LCEL)

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

from langchain_core.runnables import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma

from langchain_core.prompts import PromptTemplate

from langchain import hub
from langchain_openai import OpenAI
from langchain.agents import create_react_agent, AgentExecutor, load_tools

import os

openai_api_key = os.getenv('OPENAI_API_KEY') 

## LCEL for LLM chatbot chains

In [ ]:
model = ChatOpenAI(openai_api_key=openai_api_key)
prompt = ChatPromptTemplate.from_template("You are a skilled poet. Write a haiku about the following topic: {topic}")

# Define the chain using LCEL
chain = prompt | model

# Invoke the chain with any topic
print(print(chain.invoke({"topic": "Large Language Models"})))

## LCEL for RAG workflows

In [ ]:
# Create the retriever and model
vectorstore = Chroma.from_texts(["LangChain v0.1.0 was released on January 8, 2024."], embedding=OpenAIEmbeddings(openai_api_key=openai_api_key))
retriever = vectorstore.as_retriever()
model = ChatOpenAI(openai_api_key=openai_api_key, temperature=0)

template = """Answer the question based on the context:{context}. Question: {question}"""
prompt = ChatPromptTemplate.from_template(template)

# Create the chain and run it
chain = (
  {"context": retriever, "question": RunnablePassthrough()}
  | prompt
  | model)

chain.invoke("When was LangChain v0.1.0 released?")

# Implement functional LangChain chains

## Sequential chains with LCEL

In [ ]:
coding_prompt = PromptTemplate.from_template(
    """Write Python code to loop through the following list, printing each element: {list}""")
validate_prompt = PromptTemplate.from_template(
    """Consider the following Python code: {answer} If it doesn't use a list comprehension, update it to use one. If it does use a list comprehension, return the original code without explanation:""")

llm = ChatOpenAI(openai_api_key=openai_api_key)

# Create the sequential chain
chain = ({"answer": coding_prompt | llm | StrOutputParser()}
         | validate_prompt
         | llm 
         | StrOutputParser() )

# Invoke the chain with the user's question
chain.invoke({"list": "[3, 1, 4, 1]"})

## Passing values between chains

In [ ]:
# Make ceo_response available for other chains
ceo_response = (
    ChatPromptTemplate.from_template("You are a CEO. Describe the most lucrative consumer product addressing the following consumer need in one sentence: {input}.")
    | ChatOpenAI(openai_api_key=openai_api_key)
    | {"ceo_response": RunnablePassthrough() | StrOutputParser()}
)

advisor_response = (
    ChatPromptTemplate.from_template("You are a strategic adviser. Briefly map the outline and business plan for {ceo_response} in 3 key steps.")
    | ChatOpenAI(openai_api_key=openai_api_key)
    | StrOutputParser()
)

overall_response = (
    ChatPromptTemplate.from_messages(
        [
            ("human", "CEO response:\n{ceo_response}\n\nAdvisor response:\n{advisor_response}"),
            ("system", "Generate a final response including the CEO's response, the advisor response, and a summary of the business plan in one sentence."),
        ]
    )
    | ChatOpenAI(openai_api_key=openai_api_key)
    | StrOutputParser()
)

# Create a chain to insert the outputs from the other chains into overall_response
business_idea_chain = (
    {"ceo_response": ceo_response, "advisor_response": advisor_response}
    | overall_response
    | ChatOpenAI(openai_api_key=openai_api_key)
    | StrOutputParser()
)

print(business_idea_chain.invoke({"input": "Typing on mobile touchscreens is slow.", "ceo_response": "", "advisor_response": ""}))

# An introduction to LangChain agents

## Zero-Shot ReAct agents

In [ ]:
llm = OpenAI(model_name="gpt-3.5-turbo-instruct", temperature=0, openai_api_key=openai_api_key)

# Define the tools
tools = load_tools(["llm-math"], llm=llm)

prompt = hub.pull("hwchase17/react")

agent = create_react_agent(
    llm,
    tools,
    prompt=prompt,
)

agent_executor = AgentExecutor(
    agent=agent, tools=tools, handle_parsing_errors=True, verbose=True, max_iterations=5
)

input = (
    "What is 10 multiplied by 50?"
)
agent_executor.invoke({"input": input})